<a href="https://colab.research.google.com/github/sh-0620/dacon-recommender-system/blob/main/%08article_nouns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
data_path = '/content/drive/MyDrive/데이콘/기사추천시스템/open/'

In [4]:
article_info = pd.read_csv(data_path + 'article_info.csv')

In [5]:
view_log = pd.read_csv(data_path + 'view_log.csv')

In [8]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download pt_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 66.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
import spacy
# 영어 및 포르투갈어 모델 로드

nlp_en = spacy.load('en_core_web_sm')
nlp_pt = spacy.load('pt_core_news_sm')

In [10]:
import re
import nltk
from nltk.corpus import stopwords

# nltk 불용어 다운로드
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words_english = set(stopwords.words('english'))
stop_words_portuguese = set(stopwords.words('portuguese'))

combined_stop_words = stop_words_english.union(stop_words_portuguese)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
from nltk.stem import WordNetLemmatizer

# 표제어 처리 객체 생성
lemmatizer = WordNetLemmatizer()

# 텍스트 전처리 함수
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    words = [word for word in words if word not in combined_stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    words = [word for word in words if len(word) >= 2]

    return words

In [12]:
# 영어 명사만 추출하는 함수 정의
def extract_nouns_en(text):
    doc = nlp_en(text)
    nouns = [token.text for token in doc if token.pos_ == 'NOUN']
    return ' '.join(nouns)

# 포르투갈어 명사만 추출하는 함수 정의
def extract_nouns_pt(text):
    doc = nlp_pt(text)
    nouns = [token.text for token in doc if token.pos_ == 'NOUN']
    return ' '.join(nouns)

# 명사만 추출하는 함수 정의
def extract_nouns(row):
    if row['Language'] == 'pt':
        return extract_nouns_pt(row['Content'])
    else:
        return extract_nouns_en(row['Content'])

# 'Content' 열에서 명사만 추출하여 새로운 열 'Content_Nouns' 생성

article_info['Content_Nouns'] = article_info.apply(extract_nouns,axis =1)

# 결과 확인
print(article_info[['Content', 'Content_Nouns']])

                                                Content  \
0     I've been using git full time for the past 4 y...   
1     Intel has acquired computer vision and machine...   
2     One of the reasons AngularJS is so great to wo...   
3     Despite recent positive news and a renewed int...   
4     Last year around this time, I wrote that The B...   
...                                                 ...   
3003  Quando iniciei minha jornada em Swift, saindo ...   
3004  It's been an amazing journey to get here and i...   
3005  This is part one in a series of posts about Ko...   
3006  O Instituto Tecnológico de Aeronáutica (ITA) ,...   
3007  Amplitude announced this morning that it has r...   

                                          Content_Nouns  
0     git time years tips way article git months Tab...  
1     computer vision machine learning navigation se...  
2     reasons idea testing framework check in source...  
3     news interest investors country tsumoney reali...  
4

In [14]:
article_info

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion,Content_Nouns
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN,git time years tips way article git months Tab...
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN,computer vision machine learning navigation se...
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN,reasons idea testing framework check in source...
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN,news interest investors country tsumoney reali...
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN,year time Web re - architecture web informatio...
...,...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG,iniciei jornada versão muuuuuitos problemas ve...
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN,journey feedback thousands community members c...
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP,part series posts part design language develop...
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN,cursos meio plataforma ensino plataforma conta...


In [13]:
article_info.to_csv(data_path + 'article_nouns.csv')